In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('/content/drive/Shareddrives/CS2470/books_datasetv2.csv')
df.head

<bound method NDFrame.head of       Unnamed: 0           ISBN  \
0              0  9781419726552   
1              1  9780982301821   
2              2  9780316407076   
3              3  9780718088804   
4              4  9780451495426   
...          ...            ...   
1641        1666  9780451491640   
1642        1667  9780008263737   
1643        1668  9780525522478   
1644        1669  9780062676832   
1645        1670  9780062570604   

                                                  title  series  \
0     F*ck, That's Delicious: An Annotated Guide to ...       0   
1     10-Day Green Smoothie Cleanse: Lose Up to 15 P...       0   
2                                           15th Affair       1   
3       42 Faith: The Rest of the Jackie Robinson Story       0   
4     7 Lessons from Heaven: How Dying Taught Me to ...       0   
...                                                 ...     ...   
1641                               Archangel's Prophecy       1   
1642         

In [4]:
import re
# clean title data

def clean_data(df, col, clean_col):

    # change to lower and remove spaces on either side
    df[clean_col] = df[col].apply(lambda x: x.lower().strip())

    # remove extra spaces in between
    df[clean_col] = df[clean_col].apply(lambda x: re.sub(' +', ' ', x))

    # remove punctuation
    df[clean_col] = df[clean_col].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

   
    return df

# calling function
df = clean_data(df, 'title', 'title_clean')
df.head()

,Unnamed: 0,ISBN,title,series,author,rating,genre,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller,top5,length_of_title,top_authors,title_clean
0,0,9781419726552,"F*ck, That's Delicious: An Annotated Guide to ...",0,Action Bronson,4.17,food-and-drink,Harry N. Abrams,Hardcover,192,September,2017,1.0,0,57,0,f ck that s delicious an annotated guide to ...
1,1,9780982301821,10-Day Green Smoothie Cleanse: Lose Up to 15 P...,0,J.J. Smith,4.11,health,Adiva Publishing,Paperback,167,February,2014,1.0,0,63,0,day green smoothie cleanse lose up to p...
2,2,9780316407076,15th Affair,1,James Patterson,4.09,mystery,Hachette,Hardcover,351,May,2016,1.0,1,11,1,th affair
3,3,9780718088804,42 Faith: The Rest of the Jackie Robinson Story,0,Ed Henry,4.16,biography,Harper Collins,Hardcover,368,April,2017,1.0,1,47,0,faith the rest of the jackie robinson story
4,4,9780451495426,7 Lessons from Heaven: How Dying Taught Me to ...,0,Mary C. Neal,4.08,non-fiction,Convergent Books,Paperback,288,September,2017,1.0,0,68,0,lessons from heaven how dying taught me to ...


In [5]:
# tokenization
import nltk
nltk.download('punkt')
df["title_token"] = df.loc[:, "title_clean"].apply(nltk.word_tokenize)

# list to strings
#df["title_token"] = [','.join(map(str, l)) for l in df['title_token']]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
df["title_token"]

0       [f, ck, that, s, delicious, an, annotated, gui...
1       [day, green, smoothie, cleanse, lose, up, to, ...
2                                            [th, affair]
3       [faith, the, rest, of, the, jackie, robinson, ...
4       [lessons, from, heaven, how, dying, taught, me...
                              ...                        
1641                             [archangel, s, prophecy]
1642              [sunshine, at, the, comfort, food, caf]
1643                                  [depth, of, winter]
1644                           [the, sentence, is, death]
1645                                      [dread, nation]
Name: title_token, Length: 1646, dtype: object

In [7]:
title = df["title_token"]
label = df["NYT_bestseller"]
train_x, test_x, train_y, test_y = train_test_split(title, label, test_size=0.3)
train_title = [i for i in train_x]
train_label = [i for i in train_y]
test_title = [i for i in test_x]
test_label = [i for i in test_y]

In [8]:
def build_vocab(sentences):
	tokens = []
	for s in sentences: tokens.extend(s) 
	all_words = sorted(list(set(tokens)))  
	vocab =  {word:i for i,word in enumerate(all_words)}
	return vocab

def convert_to_id(vocab, sentences):
  id =[]
  for j in range(len(sentences)) :
    id.append([vocab[i] for i in sentences[j]])
  return id

In [9]:
vocab_all = build_vocab(title) # dictionary
id_all =  convert_to_id(vocab_all, title)

In [10]:
## vocab, id, id with padding

train_vocab = build_vocab(train_title)

train_id = convert_to_id(train_vocab, train_title)
train_id_padded = tf.keras.preprocessing.sequence.pad_sequences(train_id,padding="post")
train_id_padded = tf.cast(train_id_padded, dtype = tf.float64)
train_label = tf.cast(train_label, dtype = tf.float64)


print(train_vocab)
print(train_id_padded)

test_vocab = build_vocab(test_title)
test_id = convert_to_id(test_vocab, test_title)
test_id_padded = tf.keras.preprocessing.sequence.pad_sequences(test_id, padding="post")
test_id_padded = tf.cast(test_id_padded, dtype = tf.float64)
test_label = tf.cast(test_label, dtype = tf.float64)


{'a': 0, 'about': 1, 'absolute': 2, 'absolutely': 3, 'abundance': 4, 'abyss': 5, 'accelerations': 6, 'accidentally': 7, 'according': 8, 'account': 9, 'accout': 10, 'achieve': 11, 'acker': 12, 'action': 13, 'adam': 14, 'addiction': 15, 'addictive': 16, 'administration': 17, 'adoption': 18, 'adult': 19, 'adults': 20, 'adventure': 21, 'adventures': 22, 'adversity': 23, 'affair': 24, 'affairs': 25, 'affluence': 26, 'affordable': 27, 'afghanistan': 28, 'afraid': 29, 'african': 30, 'after': 31, 'afterlives': 32, 'again': 33, 'against': 34, 'age': 35, 'agent': 36, 'ageproof': 37, 'ages': 38, 'air': 39, 'akatsuki': 40, 'al': 41, 'alaska': 42, 'albert': 43, 'alchemist': 44, 'alert': 45, 'alex': 46, 'alexander': 47, 'alice': 48, 'alive': 49, 'all': 50, 'allegedly': 51, 'allowed': 52, 'almost': 53, 'alone': 54, 'always': 55, 'am': 56, 'amal': 57, 'amazing': 58, 'ambition': 59, 'america': 60, 'american': 61, 'among': 62, 'an': 63, 'anatomy': 64, 'ancient': 65, 'and': 66, 'andy': 67, 'anger': 68, '

In [12]:
!pip install keras-multi-head
!pip install keras-self-attention
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D, GRU, Dropout, LSTM, Bidirectional, Attention
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from collections import Counter
from sklearn.model_selection import train_test_split
from keras_self_attention import SeqSelfAttention
from keras_multi_head import MultiHead

  Created wheel for keras-multi-head: filename=keras_multi_head-0.29.0-py3-none-any.whl size=14993 sha256=8e829fc3bac4e98dd80b2f31440096ae464a182b40f7e9432e2c347fb2d84041
  Stored in directory: /root/.cache/pip/wheels/86/aa/3c/9d15d24005179dae08ff291ce99c754b296347817d076fd9fb
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=f7655403c2a4b584e5b016b1db179fbdfdac58a7cd400c52c13bd3ce99e2b79e
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-multi-head keras-self-attention


Self attention model

In [13]:
from keras.backend import flatten
tf.random.set_seed(1001)
batch_size = 100 
embedding_size = 128		
hidden_size = 64
vocab_size = len(train_id_padded)

# expand model / incorporte other types of variables / compare diff. strategies 

model = Sequential()
embedding_layer = Embedding(input_dim = 5000, mask_zero=True, output_dim = hidden_size)
model.add(embedding_layer)
model.add(Bidirectional(LSTM(units=20, return_sequences = True, dropout=0.5, recurrent_dropout=0.7)))
model.add(SeqSelfAttention(attention_activation = 'sigmoid'))
model.add(GRU(hidden_size))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 bidirectional (Bidirectiona  (None, None, 40)         13600     
 l)                                                              
                                                                 
 seq_self_attention (SeqSelf  (None, None, 40)         2625      
 Attention)                                                      
                                                                 
 gru (GRU)                   (None, 64)                20352     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 6

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(train_id_padded, train_label, epochs=5, batch_size=batch_size)

Epoch 1/5
12/12 [==============================] - 31s 191ms/step - loss: 0.6802 - accuracy: 0.6571
Epoch 2/5
12/12 [==============================] - 2s 188ms/step - loss: 0.6605 - accuracy: 0.6684
Epoch 3/5
12/12 [==============================] - 2s 195ms/step - loss: 0.6459 - accuracy: 0.6684
Epoch 4/5
12/12 [==============================] - 2s 185ms/step - loss: 0.6315 - accuracy: 0.6719
Epoch 5/5
12/12 [==============================] - 2s 196ms/step - loss: 0.6007 - accuracy: 0.7057


In [15]:
y_score = model.predict(test_id_padded)
y_score = [[1 if i >= 0.5 else 0 for i in sc] for sc in y_score] # threshold 0.5 okay?

n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == test_label[i] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy in test: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy in test: 66.60%


CNN Model

In [ ]:
model2 = Sequential()
embedding_layer = Embedding(input_dim = 5000, mask_zero=True, output_dim = hidden_size)
model2.add(embedding_layer)
model2.add(Bidirectional(LSTM(units=20, return_sequences = True, dropout=0.5, recurrent_dropout=0.7)))
model2.add(SeqSelfAttention(attention_activation = 'sigmoid'))
model2.add(GRU(hidden_size))
model2.add(Flatten())
model2.add(Dense(1,activation='sigmoid'))
model2.summary()
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model2.fit(train_id_padded, train_label, epochs=5, batch_size=batch_size)
y_score = model2.predict(test_id_padded)
y_score = [[1 if i >= 0.5 else 0 for i in sc] for sc in y_score] # threshold 0.5 okay?

n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == test_label[i] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy in test: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

multi attention

In [16]:
model3 = Sequential()
embedding_layer = Embedding(input_dim = 5000, mask_zero=True, output_dim = hidden_size)
model3.add(embedding_layer)
model3.add(MultiHead(LSTM(units=20), layer_num=5,name = 'Multi-LSTMs'))
model3.add(GRU(hidden_size))
model3.add(Flatten())
model3.add(Dense(1,activation='sigmoid'))
model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          320000    
                                                                 
 Multi-LSTMs (MultiHead)     (None, 20, 5)             34000     
                                                                 
 gru_1 (GRU)                 (None, 64)                13632     
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 367,697
Trainable params: 367,697
Non-trainable params: 0
_________________________________________________________________


In [17]:
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model3.fit(train_id_padded, train_label, epochs=5, batch_size=batch_size)
y_score = model3.predict(test_id_padded)
y_score = [[1 if i >= 0.5 else 0 for i in sc] for sc in y_score] # threshold 0.5 okay?

n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == test_label[i] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy in test: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Epoch 1/5
12/12 [==============================] - 23s 143ms/step - loss: 0.6759 - accuracy: 0.6484
Epoch 2/5
12/12 [==============================] - 2s 145ms/step - loss: 0.6434 - accuracy: 0.6684
Epoch 3/5
12/12 [==============================] - 2s 143ms/step - loss: 0.6299 - accuracy: 0.6684
Epoch 4/5
12/12 [==============================] - 2s 139ms/step - loss: 0.6124 - accuracy: 0.6684
Epoch 5/5
12/12 [==============================] - 2s 141ms/step - loss: 0.5358 - accuracy: 0.6944
Accuracy in test: 65.99%
